# Student Loan Risk with Deep Learning

In [11]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [27]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [13]:
# Review the data types associated with the columns
loans_df.dtypes

,0
payment_history,float64
location_parameter,float64
stem_degree_score,float64
gpa_ranking,float64
alumni_success,float64
study_major_code,float64
time_to_completion,float64
finance_workshop_score,float64
cohort_ranking,float64
total_loan_score,float64


In [28]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

,count
credit_ranking,
1,855
0,744


### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [29]:
# Define the target set y using the credit_ranking column

y =loans_df['credit_ranking']


# Display a sample of y
y.sample(5)


,credit_ranking
596,1
857,1
1251,0
665,0
786,0


In [30]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=['credit_ranking'])

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [31]:
X = loans_df.drop(columns=['credit_ranking'])
y = loans_df['credit_ranking']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Assign the function a random_state equal to 1
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((1279, 11), (320, 11), (1279,), (320,))

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [32]:
# Create a StandardScaler instance

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit the scaler to the features training dataset

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the scaler to the features training dataset
pd.DataFrame(X_train_scaled, columns=X.columns).head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,-0.733079,0.664893,-1.257044,-0.320459,-0.453622,-0.742407,-0.645507,0.240001,0.988460,0.063095,-0.872234
1,1.067741,-0.623462,1.523148,0.608863,-0.369546,-1.125190,-1.112003,0.187899,-1.753513,-0.173904,-0.779785
2,-1.746040,-1.071585,-1.358142,-0.534917,-0.789922,1.075808,0.536281,-2.677737,2.327564,0.774090,3.287990
3,-0.620528,0.496847,-1.054849,-0.034513,-0.201396,0.118853,1.189375,0.375468,1.243528,-0.766400,-0.687335
4,-0.507977,0.608877,-1.004300,-0.534917,0.261018,-0.742407,-0.707707,-0.333126,-0.095575,-0.470152,-0.779785


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [24]:
# Define the the number of inputs (features) to the model
number_of_features = X_train.shape[1]


# Review the number of features
number_of_features


11

In [33]:
# Define the number of inputs (features) to the model
number_of_features = X_train.shape[1]

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = (2 * number_of_features) // 3 + 1

# Define the number of hidden nodes for the second hidden layer

hidden_nodes_layer2 = hidden_nodes_layer1 // 2  # Typically fewer than the first layer

# Define the number of neurons in the output layer
output_neurons = 1

# Display the values to confirm
hidden_nodes_layer1, hidden_nodes_layer2, output_neurons

(8, 4, 1)

In [34]:
# Create the Sequential model instance

model = Sequential()
# Add the first hidden layer
model.add(Dense(units=16, input_dim=number_of_features, activation='relu'))


# Add the second hidden layer
model.add(Dense(units=8, activation='relu'))


# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=1, activation='sigmoid'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Display the Sequential model summary


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Compile the Sequential model
# Define the model architecture
model = Sequential()
model.add(Dense(units=16, input_dim=number_of_features, activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4469 - loss: 0.7489 - val_accuracy: 0.5195 - val_loss: 0.7059
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5157 - loss: 0.6979 - val_accuracy: 0.5742 - val_loss: 0.6684
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5730 - loss: 0.6563 - val_accuracy: 0.6016 - val_loss: 0.6421
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6132 - loss: 0.6300 - val_accuracy: 0.6367 - val_loss: 0.6229
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6523 - loss: 0.6153 - val_accuracy: 0.6367 - val_loss: 0.6085
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6727 - loss: 0.6011 - val_accuracy: 0.6328 - val_loss: 0.5957
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6948 - loss: 0.6034 - val_accuracy: 0.6602 - val_loss: 0.5853
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7138 - loss: 0.5683 - val_accuracy: 0.6680 - val_loss:

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [40]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)



# Display the model loss and accuracy results
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7583 - loss: 0.4976 
Test Loss: 0.5219448208808899
Test Accuracy: 0.7406250238418579


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [41]:
# Set the model's file path
file_path = "student_loans.keras"



# Export your model to a keras file
model.save(file_path)



---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [42]:
# Set the model's file path

from tensorflow.keras.models import load_model

# Load the model to a new object
model = load_model("student_loans.keras")


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [47]:
# Make predictions with the test data
predictions = model.predict(X_test_scaled)

# Display a sample of the predictions
print(predicted_classes[:10])  # Display the first 10 predictions

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
[[0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]]


In [45]:
# Save the predictions to a DataFrame and round the predictions to binary results
# Convert predictions to binary results (1 for success, 0 for failure)
predicted_classes = (predictions > 0.5).astype(int)

# Save the predictions to a DataFrame
predictions_df = pd.DataFrame(predicted_classes, columns=["predictions"])

# Display the first few rows of the DataFrame
predictions_df.head()

,predictions
0,0
1,0
2,1
3,1
4,1


### Step 4: Display a classification report with the y test data and predictions

In [48]:
# Print the classification report with the y test data and predictions
from sklearn.metrics import classification_report

print(classification_report(y_test, predicted_classes))


              precision    recall  f1-score   support

           0       0.72      0.76      0.74       154
           1       0.76      0.72      0.74       166

    accuracy                           0.74       320
   macro avg       0.74      0.74      0.74       320
weighted avg       0.74      0.74      0.74       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

Answer :

To build a recommendation system for student loan options, you would need the following data:

- Student Profile Data: Includes GPA, degree type, major, school ranking, and financial need. This information helps determine the student's eligibility and potential for loan repayment.

- Financial Data: Includes income level, current debts, and expected post-graduation salary, as these directly influence loan repayment capacity.

- Loan Product Data: Details of different loan types (interest rates, repayment terms, grace periods). This allows the recommendation system to align student needs with appropriate loan products.

- Health Data: Both physical and mental health can impact a student’s ability to complete their studies on time. Including this data allows the system to offer loans with flexible terms or deferment options for students who may need additional time due to health-related challenges.


2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

Answer:
For this system, context-based filtering would be most appropriate. This is because context-based filtering uses the specific attributes of each student and their financial and academic profile to recommend loan products that best fit their unique situation. Content-based filtering could also work if focused on matching specific loan features to students' needs, but context-based filtering would provide the broader context of students' personal circumstances.



**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
answer:

Privacy and Data Security: Handling sensitive data like income, debt, and academic records requires strict security and privacy measures. Unauthorized access or data leaks could harm students’ financial safety and privacy.

Data Accuracy and Completeness: Missing or inaccurate data can lead to poor recommendations. For example, if a student's income or financial aid information is outdated, the recommendation might suggest loans they cannot afford, potentially leading to financial hardship.
